In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import colors
import pickle
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import sklearn.cross_validation
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
%matplotlib inline
import matplotlib as mpl
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from collections import Counter 
import _pickle as cPickle
import random

In [2]:
def downoad_data(N,name_file):
    path = '..\output\X' + str(N) + '\\'
    path_name_file = str(path) + str(name_file) + '.pkl'
    file = pd.read_pickle(path_name_file)
    return file 
def upload_data(N,file,name_file):
    path = '..\output\X' + str(N) + '\\'
    path_name_file_plk = str(path) + str(name_file) + '.pkl'
    path_name_file_csv = str(path) + str(name_file) + '.csv'    
    file.to_pickle(path_name_file_plk)
    file.to_csv(path_name_file_csv, sep='\t', encoding='utf-8')
    return path_name_file_plk 

In [3]:
def linear_regression_parameter(X, test_SIZE):
# Use only one feature
    X_input = X[['lenght_utterance','distance_to_end','median_duration']]
    Y_output = X['duration']

# Split the data into training/testing sets
# Split the targets into training/testing sets

    if test_SIZE == 0:
        X_train = X_input
        Y_train = Y_output
    else:    
        X_train, X_test, Y_train, Y_test = sklearn.cross_validation.train_test_split(X_input, Y_output, test_size = test_SIZE, random_state = 5)


# Create linear regression object
    regr = linear_model.LinearRegression()

# Train the model using the training sets
    regr.fit(X_train, Y_train)
    parameters = pd.DataFrame(list(zip(X_input.columns,regr.coef_)), columns= ['features','estimate_coefficients'])
    print(parameters)

    if test_SIZE == 0:
        return regr,parameters, X_train, Y_train
    else:
        
    
    
    # Make predictions using the testing set
        Y_pred = regr.predict(X_test) 

# The coefficients
# The mean squared error
        print("\nMean squared error: %.3f" % mean_squared_error(Y_test, Y_pred))
# Explained variance score: 1 is perfect prediction
        print('\nVariance score: %.2f' % r2_score(Y_test, Y_pred))

# Plot outputs
        Y_test=pd.DataFrame(Y_test)
        result = pd.concat([ Y_test, X_test], axis=1)
        index_selected = result.index.values
        df = pd.DataFrame({'index': index_selected, 'predicted_duration': Y_pred})
        df = df.set_index('index')
        result = pd.concat([ df, result], axis=1)
        result['distance_to_end'] = pd.to_numeric(result['distance_to_end'], errors='coerce').fillna(0)
        result['lenght_utterance'] = pd.to_numeric(result['lenght_utterance'], errors='coerce').fillna(0)
    
        result.plot(x = 'distance_to_end', y = 'predicted_duration', kind = 'scatter', grid=False,  alpha = 0.15,color='black', figsize= (15,10))
        result.plot(x = 'distance_to_end', y = 'duration', kind = 'scatter', grid=False,  alpha = 0.15,color='blue', figsize= (15,10))
        plt.xlabel("Distance utterance to the end (word)")
        plt.ylabel("Duration(s) ")
        plt.title("Expected & real duration vs distance utterance to the end ")
        plt.show()
    
        return regr,parameters, result, X_train, Y_train


In [9]:
def speech_rate_one_conversation(X_processed, id_conv, id_speaker, word_to_exclude_expected, word_to_exclude_real):
    
    X_filtered_expected = X_processed[(~X_processed['word'].isin(word_to_exclude_expected))]
    X_filtered_expected = X_filtered_expected[(X_filtered_expected['ID_conversation'] == id_conv) & (X_filtered_expected['ID_speaker'] == id_speaker)]
    
    X_filtered_real = X_processed[(~X_processed['word'].isin(word_to_exclude_real))]
    X_filtered_real = X_filtered_real[(X_filtered_real['ID_conversation'] == id_conv) & (X_filtered_real['ID_speaker'] == id_speaker)]
    
    id_utterance = X_filtered_real['ID_utterance'].unique()
    for indice_id in id_utterance:
        X_filtered_utterance = X_filtered_real[X_filtered_real['ID_utterance'] == indice_id]
    
        codition_init = True
        condition_dataframe_not_empty = True
        while codition_init & condition_dataframe_not_empty:
            if X_filtered_utterance['word'].iloc[0] in word_to_exclude_expected:
                indici = X_filtered_utterance.index
                c = indici[0]
                X_filtered_real.drop(c, inplace=True)
                X_filtered_utterance.drop(c, inplace=True)
                condition_dataframe_not_empty = ~X_filtered_utterance.empty
            else:
                codition_init = False
            
        codition_init = True
        condition_dataframe_not_empty = ~X_filtered_utterance.empty
        while codition_init & condition_dataframe_not_empty:
            if X_filtered_utterance['word'].iloc[-1] in word_to_exclude_expected:
                indici = X_filtered_utterance.index
                c = indici[-1]
                X_filtered_real.drop(c, inplace=True)
                X_filtered_utterance.drop(c, inplace=True)
                condition_dataframe_not_empty = ~X_filtered_utterance.empty
            else:
                codition_init = False       
               
    
    
    real_duration = X_filtered_real.groupby(['ID_utterance'])['duration'].sum()
   # print(real_duration)
    expected_duration = X_filtered_expected.groupby(['ID_utterance'])['expected_duration'].sum()
   # print(expected_duration)
    pointwise_speech_rate = real_duration/expected_duration
    #print(pointwise_speech_rate)
    mean_log_pointwise_speech_rate = (np.log(pointwise_speech_rate)).mean() 
    mean_pointwise_speech_rate = pointwise_speech_rate.mean()
    
    return [id_conv, id_speaker, mean_log_pointwise_speech_rate, mean_pointwise_speech_rate]    

In [10]:
def compute_speech_rate(n_min_words, X_regr, word_to_exclude_expected, word_to_exclude_real, n_directory):
    
    X = downoad_data(n_directory,'X')
    #X_regr = downoad_data(n_directory_regression,'X')
    [regr,parameters,X_train, Y_train]= linear_regression_parameter(X_regr, 0)
    X['expected_duration'] = regr.predict(X[['lenght_utterance','distance_to_end','median_duration']])
    
    X_processed1 = X[(~X['word'].isin(word_to_exclude_expected))]

    n_significative_utterance = X_processed1.groupby(['ID']).size()
    df = pd.DataFrame(n_significative_utterance, columns = {'n_word_significative'})
    indici_uterance_significative = df[df['n_word_significative'] > n_min_words  ].index
    X_processed_new = X[(X['ID'].isin(indici_uterance_significative))]
    X_processed_new.index = np.arange(0, len(X_processed_new))

    
    list_of_speech_rate = []
    conv_tab = pd.read_csv("..\conv_tab.csv")
    indici = conv_tab['id_conv'].as_matrix()
    indicistr = []
    for i in indici:
        indicistr.append(str(i))
    
    progress = 0

    for id_conv in indicistr:

        progress = progress + (1/len(indicistr))
        update_progress(progress/100)
        for id_speaker in ['A','B']:
            [id_conv, id_speaker, mean_log_pointwise_speech_rate,mean_pointwise_speech_rate] = speech_rate_one_conversation(X_processed_new,id_conv, id_speaker, word_to_exclude, word_to_exclude_real)
            list_of_speech_rate.append([id_conv, id_speaker, mean_log_pointwise_speech_rate,mean_pointwise_speech_rate])
        
            
    speech_rate_speakers = pd.DataFrame(list_of_speech_rate, columns=['id_conv', 'id_caller', 'm_log_pointw_speech_rate','mean_pointwise_speech_rate'])

    caller_tab = pd.read_csv("..\caller_tab.csv")
    call_con_tab = pd.read_csv("..\call_con_tab.csv")
    topic_tab = pd.read_csv("..\Topic_tab.csv")
    call_con_tab_selected = call_con_tab[['id_conv','id_caller','id_speaker','id_topic']]
    caller_tab_selected = caller_tab[['id_speaker','sex','age','geography','level_study']]
    
    speaker_tab = call_con_tab_selected.merge(caller_tab_selected, how='left', on = 'id_speaker')
    
    speech_rate_speakers['id_conv'] = speech_rate_speakers['id_conv'].astype(str).astype(int)

    
    speech_rate = pd.merge(speech_rate_speakers,speaker_tab, how='left', on = ['id_conv', 'id_caller']  )
    speech_rate.drop_duplicates()
    speaker_B = speech_rate[['id_conv','id_caller', 'id_speaker']]
    speaker_B['id_caller'] = speaker_B['id_caller'].replace(['A'], 1)
    speaker_B['id_caller'] = speaker_B['id_caller'].replace(['B'], 0)
    speaker_B = speaker_B.sort_values(by = ['id_conv','id_caller'], ascending = [True,True])
    speaker_B['id_caller'] = speaker_B['id_caller'].replace([1], 'A')
    speaker_B['id_caller'] = speaker_B['id_caller'].replace([0], 'B')
    

    speaker_B.index = speech_rate.index
    speech_rate = pd.merge(speech_rate, speaker_B, left_index=True, right_index=True)

    del speech_rate['id_conv_y']
    del speech_rate['id_caller_y']

    speech_rate.rename(columns={'id_conv_x': 'id_conv', 'id_caller_x': 'id_caller', 'id_speaker_y': 'id_interlocutor', 'id_speaker_x': 'id_speaker'}, inplace=True)
    speech_rate['age'] = 1991 - speech_rate['age']
    
    name_file = 'speech_rate_minword' + str(n_min_words) 
    upload_data(n_directory,speech_rate,name_file)
    
    return speech_rate
        

In [6]:
import time, sys

# update_progress() : Displays or updates a console progress bar
## Accepts a float between 0 and 1. Any int will be converted to a float.
## A value under 0 represents a 'halt'.
## A value at 1 or bigger represents 100%
def update_progress(progress):
    barLength = 20 # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done...\r\n"
    block = int(round(barLength*progress))
    text = "\rPercent: [{0}] {1}% {2}".format( "="*block + " "*(barLength-block), progress*100, status)
    sys.stdout.write(text)
    sys.stdout.flush()

In [21]:
  # STANDARDIZE
def standardize_speech_rate(n_min_words, n_directory):
    from sklearn import preprocessing
    
    name_file = 'speech_rate_minword' + str(n_min_words)
    speech_rate = downoad_data(n_directory,name_file)
    speech_rate_no_nan = speech_rate[np.isfinite(speech_rate['m_log_pointw_speech_rate'])]
    speech_rate_std = speech_rate_no_nan
    x = speech_rate_std[['age','m_log_pointw_speech_rate','id_conv']].values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-3,3.1))
    #x_scaled = min_max_scaler.fit_transform(x)
    x_scaled = preprocessing.scale(x)
    numeric_speech_rate_std = pd.DataFrame(x_scaled, index=speech_rate_std.index)
    numeric_speech_rate_std.columns = ['age', 'm_log_pointw_speech_rate','id_conv']
    speech_rate_std['age_std']  = numeric_speech_rate_std['age']
    speech_rate_std['m_log_pointw_speech_rate']  = numeric_speech_rate_std['m_log_pointw_speech_rate']
    #speech_rate_std['m_log_pointw_speech_rate'].plot( grid=False,  alpha = 0.3, figsize= (20,15))
    
    name_file = 'speech_rate_minword' + str(n_min_words) + 'std' 
    upload_data(n_directory,speech_rate_std, name_file)
    return speech_rate_std


In [16]:
def filter_speech_rate(N_conv_min_speaker,n_min_words, size_data, n_directory):
    
    name_file = 'speech_rate_minword' + str(n_min_words) + 'std'
    speech_rate_std = downoad_data(n_directory,name_file)
    
    speech_rate_std = speech_rate_std[speech_rate_std['id_topic'] != ' UNK']
    #count_speakerA = speech_rate_std.groupby('id_speaker').count()
    #count_speakers_A = count_speakerA[count_speakerA['id_conv'] >= N_conv_min_speaker].index
    #count_speakerB = speech_rate_std.groupby('id_interlocutor').count()
    #count_speakers_B = count_speakerA[count_speakerA['id_conv'] >= N_conv_min_speaker].index
    #speech_rate_filter = speech_rate_std[(speech_rate_std['id_speaker'].isin(count_speakers_A)) & (speech_rate_std['id_interlocutor'].isin(count_speakers_B))  ]
    
    conv_dati = pd.read_csv("..\conv_tab.csv")
    conv_dati = conv_dati[conv_dati['id_topic'] != ' UNK']
    
    if size_data == 1:
        pass
    elif (size_data <= 0.5) or (size_data > 1):
        print('\nError: The size data factor should be in th einterval [0.5, 1]')
    else: 
        N_row_to_drop = int(len(conv_dati) * (1 - size_data) )
        id_conv_to_drop = random.sample(set(conv_dati['id_conv']), N_row_to_drop)
        conv_dati = conv_dati[-conv_dati['id_conv'].isin(id_conv_to_drop)]
                   # df[~df["column"].isin(["value"])]
    speakers = list(conv_dati['speaker_A'].values)
    speakers += list(conv_dati['speaker_B'].values)
    c=Counter(speakers) 

    ID_SPEAKER = list()
    COUNT_CONV = list()

    for id_speaker in list(set (speakers)):
        ID_SPEAKER.append(id_speaker)
        COUNT_CONV.append(c[id_speaker])

    frame = [('ID_speakers', ID_SPEAKER),
         ('number_conv', COUNT_CONV),
         ]    
    conv_per_speakers_df = pd.DataFrame.from_items(frame)
    list_speakers = conv_per_speakers_df[conv_per_speakers_df['number_conv'] >= N_conv_min_speaker].ID_speakers
    
    speech_rate_filter = speech_rate_std[(speech_rate_std['id_speaker'].isin(list_speakers)) & (speech_rate_std['id_interlocutor'].isin(list_speakers))  ]

    
    name_file = 'speech_rate_minword' + str(n_min_words) + 'std_' + 'size%_'+ str(size_data*100)+ '_' + str(N_conv_min_speaker) + 'conv_speaker'
    upload_data(n_directory, speech_rate_filter, name_file)
    return speech_rate_filter
